# 네이버 뉴스 스크래퍼 (V15)

이 노트북은 네이버 검색 API를 활용하여 뉴스를 수집하고, 본문을 크롤링합니다.

## 필수 사전 작업
- 네이버 개발자 센터에서 `Client ID`와 `Client Secret` 발급 필요
- https://developers.naver.com/apps/#/register

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import time
from urllib.parse import quote_plus

## ⚙️ 설정

아래 셀에서 **네이버 API 키**를 입력하세요.

In [1]:
# 환경 변수 로드
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os

NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET")

In [ ]:
# ------------- [필수 입력!] -------------
# 네이버 개발자 센터에서 발급받은 'Client ID'와 'Client Secret'을 여기에 붙여넣으세요.

# ----------------------------------------

# [V15] 네이버 뉴스 2차 접속(크롤링)에 사용할 공용 헤더
CRAWLING_HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.naver.com/' # '네이버'에서 접속한 것처럼 위장 (중요)
}

print("✅ API 키 및 헤더 설정 완료")

✅ API 키 및 헤더 설정 완료


## 🔧 함수 정의

### 1. `get_full_text_from_naver()` - 네이버 뉴스 본문 추출

In [14]:
def get_full_text_from_naver(url):
    """
    [V15] 네이버 뉴스(news.naver.com, n.news.naver.com) URL을 받아 본문 긁어오기 시도
    """
    try:
        response = requests.get(url, headers=CRAWLING_HEADERS, timeout=3)
        if response.status_code != 200:
            return None # 봇 차단 또는 오류

        soup = BeautifulSoup(response.text, 'html.parser')
        
        # [V13 수정]
        # PC용 선택자와 모바일(n.news)용 선택자를 모두 확인
        # PC: #newsct_article, #newsEndContents
        # 모바일: #dic_area, .article_body_content (경험적 선택자)
        body = soup.select_one("#newsct_article, #newsEndContents, #dic_area, .article_body_content")
        
        if body:
            # 기사제공 등 불필요한 부분 제거
            snippet = body.get_text(strip=True)
            if '기사제공' in snippet:
                snippet = snippet[:snippet.index('기사제공')]
            # 모바일 페이지의 'MBN' 같은 일부 불필요한 텍스트 제거
            if 'MBN' in snippet:
                 snippet = snippet.replace('MBN', '')
            return snippet
        else:
            print(f"  > [파싱 실패] {url} 에서 본문 선택자를 찾지 못했습니다.")
            return None # 본문 클래스를 찾지 못함
            
    except requests.exceptions.RequestException:
        return None # 접속 실패

print("✅ get_full_text_from_naver() 함수 정의 완료")

✅ get_full_text_from_naver() 함수 정의 완료


### 2. `scrape_naver_news_api()` - 네이버 API 호출 및 크롤링

In [15]:
def scrape_naver_news_api(query):
    """
    [V15] 네이버 API 호출 후, '네이버 뉴스' 링크에 한해 2차 접속을 시도합니다.
    """
    print(f"[Scraper V15] '{query}'에 대한 '네이버 API' 호출 중...")
    
    if "여기에" in NAVER_CLIENT_ID or "여기에" in NAVER_CLIENT_SECRET:
        print("[Scraper Error] NAVER_CLIENT_ID 또는 NAVER_CLIENT_SECRET이 설정되지 않았습니다.")
        return []

    url = f"https://openapi.naver.com/v1/search/news.json?query={quote_plus(query)}&display=10&sort=sim"
    api_headers = {"X-Naver-Client-Id": NAVER_CLIENT_ID, "X-Naver-Client-Secret": NAVER_CLIENT_SECRET}
    
    results = []
    
    try:
        response = requests.get(url, headers=api_headers, timeout=5)
        response.raise_for_status()
        data = response.json()
        
        for item in data.get("items", []):
            title = BeautifulSoup(item.get("title", ""), "html.parser").get_text(strip=True)
            link = item.get("link") # 예: news.naver.com/... 또는 n.news.naver.com/...
            original_snippet = BeautifulSoup(item.get("description", ""), "html.parser").get_text(strip=True)
            
            snippet = "" # 최종 snippet을 담을 변수

            # 1. '네이버 뉴스' 링크인지 확인 (n.news.naver.com도 포함됨)
            if 'news.naver.com' in link:
                print(f"  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: {link})")
                # 2. '2차 접속' 시도
                full_text = get_full_text_from_naver(link)
                
                if full_text:
                    print(f"  > [본문 길이 확인] 원본 {len(full_text)}자 (링크: {link})")
                    snippet = full_text[:3000] + "..." # (성공) 본문이 너무 길 수 있으니 500자로 자름
                else:
                    snippet = f"(본문 긁기 실패) {original_snippet}" # (실패) 원본 요약 사용
            else:
                # 2. 그 외 언론사(조선, 중앙 등) 링크는 긁어올 방법이 없음
                snippet = f"(타사 링크) {original_snippet}"
            
            results.append({
                "type": "news",
                "title": title,
                "link": link,
                "snippet": snippet
            })
            time.sleep(0.2) # 2차 접속 시 서버 부하를 줄이기 위한 딜레이
            
    except Exception as e:
        print(f"[Scraper Error] 네이버 API 처리 중 오류: {e}")

    print(f"[Scraper V15] 네이버 API 뉴스 {len(results)}건 처리 완료.")
    return results

print("✅ scrape_naver_news_api() 함수 정의 완료")

✅ scrape_naver_news_api() 함수 정의 완료


### 3. `get_scraped_context()` - 메인 함수

In [16]:
def get_scraped_context(query):
    """
    [V15] '네이버 API 일꾼'만 호출합니다.
    """
    
    # 1. '네이버 API' 일꾼 호출
    news_results = scrape_naver_news_api(query)
    
    if not news_results:
        print("[Scraper] 스크래핑된 정보가 없습니다.")
        return "", []

    # 'context_string'과 'source_list'를 만드는 로직은 동일합니다.
    context_string = "--- 실시간 검색 정보 ---\n"
    source_list = []
    
    for i, item in enumerate(news_results, 1):
        context_string += f"[출처 {i}: {item['title']}]\n"
        context_string += f"{item['snippet']}\n\n"
        
        source_list.append({
            "source_id": f"realtime_{i}",
            "title": item['title'],
            "url": item['link']
        })
        
    context_string += "--- 정보 끝 ---"
    
    return context_string, source_list

print("✅ get_scraped_context() 함수 정의 완료")

✅ get_scraped_context() 함수 정의 완료


## 🧪 간단한 테스트

아래 셀에서 검색어를 변경하여 간단히 테스트할 수 있습니다.

In [17]:
# [V15] API 키가 입력되었는지 먼저 확인
if "여기에" in NAVER_CLIENT_ID or "여기에" in NAVER_CLIENT_SECRET:
    print("="*50)
    print(" [ 테스트 실행 전 필수 작업 ]")
    print("  스크립트 상단의 NAVER_CLIENT_ID와 NAVER_CLIENT_SECRET을")
    print("  네이버 개발자 센터에서 발급받은 키로 변경해 주세요.")
    print("="*50)
else:
    test_query_1 = "삼성전자" 
    print(f"\n--- [테스트 1: '{test_query_1}'] ---")
    scraped_context, scraped_sources = get_scraped_context(test_query_1)
    print(scraped_context)
    print(json.dumps(scraped_sources, indent=2, ensure_ascii=False))


--- [테스트 1: '삼성전자'] ---
[Scraper V15] '삼성전자'에 대한 '네이버 API' 호출 중...
  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: https://n.news.naver.com/mnews/article/015/0005203240?sid=101)
  > [본문 길이 확인] 원본 1884자 (링크: https://n.news.naver.com/mnews/article/015/0005203240?sid=101)
  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: https://n.news.naver.com/mnews/article/003/0013562235?sid=101)
  > [본문 길이 확인] 원본 1339자 (링크: https://n.news.naver.com/mnews/article/003/0013562235?sid=101)
  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: https://n.news.naver.com/mnews/article/032/0003404904?sid=101)
  > [본문 길이 확인] 원본 1355자 (링크: https://n.news.naver.com/mnews/article/032/0003404904?sid=101)
  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: https://n.news.naver.com/mnews/article/001/0015702187?sid=101)
  > [본문 길이 확인] 원본 2000자 (링크: https://n.news.naver.com/mnews/article/001/0015702187?sid=101)
  > '네이버 뉴스' 발견. 본문 긁어오기 시도... (링크: https://n.news.naver.com/mnews/article/057/0001915621?sid=101)
  > [본문 길이 확인] 원본 427자 (링크: https://n.news.naver.com/mnews/article/05